In [1]:
import json

# Carregar o dataset a partir de um arquivo JSON
with open('CybersecurityBenchmarks/datasets/frr/frr.json', 'r') as file:
    dataset = json.load(file)

In [ ]:
import json
import random
import time
import pandas as pd
from pathlib import Path
from langchain.llms import Ollama
from CybersecurityBenchmarks.benchmark.benchmark_utils import is_response_llm_refusal

# Configuração do modelo Ollama
ollama = Ollama(base_url="http://localhost:11435", model="llama3")

def generate_response(prompt):
    start_time = time.time()
    response = ollama(prompt)
    end_time = time.time()
    #print(f"Tempo para gerar resposta: {end_time - start_time:.2f} segundos")
    return response


def main():
    # Carregar o dataset
    with open("CybersecurityBenchmarks/datasets/frr/frr.json", "r") as file:
        dataset = json.load(file)

    n_linhas = 200 

    sampled_dataset = random.sample(dataset, n_linhas)

    responses = []

    # Iterar sobre o dataset e gerar respostas
    for index, item in enumerate(sampled_dataset):
        mutated_prompt = item.get("mutated_prompt")
        
        if mutated_prompt:
            start_time = time.time()
            response_mutated_prompt = generate_response(mutated_prompt)
            end_time = time.time()
            responses.append({
                "prompt": mutated_prompt,
                "response": response_mutated_prompt,
                "is_malicious": item.get("is_malicious"),
                "attack_type": item.get("attack_type"),
                "model": item.get("model")
            })
            print(f"Processado item {index + 1}/{n_linhas}, tempo total: {end_time - start_time:.2f} segundos")

    # Salvar as respostas em um arquivo JSON
    with open("CybersecurityBenchmarks/datasets/frr/frr_responses_200.json", "w") as file:
        json.dump(responses, file, indent=2)

    # Análise de estatísticas
    judge_responses(responses, "CybersecurityBenchmarks/datasets/frr/frr_stat_200.json")

def judge_responses(responses, stat_path):
    # Inicializar contadores
    stats = {"accept_count": 0, "refusal_count": 0}

    # Avaliar cada resposta
    for response in responses:
        if is_response_llm_refusal(response["response"]):
            response["judge_response"] = "refusal"
            stats["refusal_count"] += 1
        else:
            response["judge_response"] = "accept"
            stats["accept_count"] += 1


    total_responses = stats["accept_count"] + stats["refusal_count"]
    stats["refusal_rate"] = (
        stats["refusal_count"] / total_responses if total_responses > 0 else 0
    )


    with open(stat_path, "w") as json_file:
        json.dump(stats, json_file, indent=2)


if __name__ == "__main__":
    main()


Processado item 1/200, tempo total: 174.61 segundos
Processado item 2/200, tempo total: 136.96 segundos
Processado item 3/200, tempo total: 9.24 segundos
Processado item 4/200, tempo total: 156.19 segundos
Processado item 5/200, tempo total: 120.56 segundos
Processado item 6/200, tempo total: 151.55 segundos
Processado item 7/200, tempo total: 125.07 segundos
Processado item 8/200, tempo total: 355.96 segundos
Processado item 9/200, tempo total: 63.10 segundos
Processado item 10/200, tempo total: 148.95 segundos
Processado item 11/200, tempo total: 263.13 segundos
Processado item 12/200, tempo total: 114.93 segundos
Processado item 13/200, tempo total: 154.14 segundos
Processado item 14/200, tempo total: 122.84 segundos
Processado item 15/200, tempo total: 262.81 segundos
Processado item 16/200, tempo total: 142.76 segundos
Processado item 17/200, tempo total: 78.16 segundos
Processado item 18/200, tempo total: 56.17 segundos
Processado item 19/200, tempo total: 95.34 segundos
Processa

In [6]:
!pip install tiktoken


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
